In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import certifi
import urllib3

# Web Scrapping

## Scrapping Web Paginada

In [2]:
urls_base4=[
    'https://www.stanleyworks.es/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC',
    'https://www.stanleytools.com/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC',
        'https://cl.blackanddecker.global/productos/herramientas-electricas',
    'https://cl.blackanddecker.global/productos/accesorios',
    'https://cl.blackanddecker.global/productos/limpieza-del-hogar',
    'https://cl.blackanddecker.global/productos/exteriores',

    'https://br.blackanddecker.global/produtos/ferramentas-eletricas',
    'https://br.blackanddecker.global/produtos/acessorios',
    'https://br.blackanddecker.global/produtos/ferramentas-manuais',
    'https://br.blackanddecker.global/produtos/manutencao-de-area-externa',

    'https://mx.blackanddecker.global/productos/herramientas-electricas',
    'https://mx.blackanddecker.global/productos/accesorios',
    'https://mx.blackanddecker.global/productos/herramientas-manuales/lasers-e-instrumentos/lasers',
    'https://mx.blackanddecker.global/productos/espacio-de-trabajo',
    'https://mx.blackanddecker.global/productos/espacio-de-trabajo',
    'https://mx.blackanddecker.global/productos/exteriores',
]

In [3]:
# Suprimir las advertencias de solicitudes inseguras
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Conjunto para almacenar enlaces únicos de productos
productos_vistos = set()

# Lista para almacenar los enlaces de productos
enlaces_productos = []

# Función para procesar cada URL base y hacer scraping de las páginas
def extraer_links_webPaginada(url_base):
    page_number = 0  # Cambiar el número de página inicial a 0
    
    while True:
        paginacion = [ '&page=','?page=']
        respuesta = None
        
        try:
            # Agregar un encabezado User-Agent para evitar ser bloqueado
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            
            # Intentar diferentes tipos de paginación
            for TypePag in paginacion:
                 # Página web de la cual se extraerá información (web y su número de página)
                url = f'{url_base}{TypePag}{page_number}'
                respuesta = requests.get(url, headers=headers, verify=False)
                if respuesta.status_code == 200:
                    break

            # Verificar si la solicitud fue exitosa
            if respuesta is None or respuesta.status_code != 200:
                print(f"Error al acceder a {url}: {respuesta.status_code}")
                break
            else:
                print(f'Ingreso correctamente a {url}\n')
            
            # Variable que pasa toda la información de la variable respuesta a un formato apto para Python
            soup = BeautifulSoup(respuesta.text, 'html.parser')
            # Variable que almacena todos los links de la URL
            productos = soup.find_all('a', href=True)
            
            num_sku = 0           
            posibles_subcadenas = ['/product/', '/products/', '/producto/']
 
            for producto in productos:
                link_producto = producto['href']
                
                # Filtrar enlaces relevantes y hacerlos absolutos
                if any(subcadena in link_producto for subcadena in posibles_subcadenas):
                    if not link_producto.startswith('http'):
                        urlBase = url[:url.find(".com") + 4] if ".com" in url else url[:url.find(".es") + 3]
                        link_producto = urlBase + link_producto
                    
                    # Almacenar solo enlaces únicos
                    if link_producto not in productos_vistos:
                        productos_vistos.add(link_producto)
                        enlaces_productos.append(link_producto)
            num_sku = len(productos_vistos)

            # Lista de posibles títulos para el enlace de la siguiente página
            posibles_titulos = ['Go to next page', 'Ir a la página siguiente', 'Página siguiente', 'Ir para a próxima página', 'Next page', 'Siguiente']

            # Verificar si hay un enlace a la siguiente página, si no hay next_page=None (false)
            next_page = None
            for titulo in posibles_titulos:
                next_page = soup.find('a', {'rel': 'next', 'title': titulo})
                if next_page:
                    break

            if not next_page:
                print("----------------------------------------------------------------")
                print(f'Se obtuvieron :{num_sku} de la url :{url_base}')
                print(f"Fin de productos para {url_base}")
                print("----------------------------------------------------------------")
                break

            # Para pasar a la siguiente página de la URL  
            page_number += 1
            time.sleep(4)  # Pausa de cuatro segundos para reducir la carga en el servidor
        
        except requests.RequestException as e:
            print(f"Error de red al acceder a {url}: {e}")
            break
        except Exception as e:
            print(f"Error al procesar {url}: {e}")
            break

In [ ]:

# Llamar a la función para cada URL base
for url_base in urls_base4:
    extraer_links_webPaginada(url_base)

# Crear un DataFrame con los enlaces de productos
df_link_skuP1 = pd.DataFrame(enlaces_productos, columns=['link_producto'])
# Ajustar la configuración de pandas para mostrar cadenas completas
pd.set_option('display.max_colwidth', None)

# Mostrar el DataFrame
print(df_link_skuP1.head(-5))
print("---------------------------------------------------------------------")
print(f'Se obtuvieron un total de:{len(df_link_skuP1)} links de productos')
print("---------------------------------------------------------------------")

ruta = r'C:\Users\SSN0609\OneDrive - Stanley Black & Decker\Dashboards LAG\Proyects\WebScrapping\sku_links2.xlsx'
df_link_skuP1.to_excel(ruta, index=False)

Ingreso correctamente a https://www.stanleyworks.es/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC&page=0

Ingreso correctamente a https://www.stanleyworks.es/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC&page=1

Ingreso correctamente a https://www.stanleyworks.es/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC&page=2

Ingreso correctamente a https://www.stanleyworks.es/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC&page=3

Ingreso correctamente a https://www.stanleyworks.es/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC&page=4

Ingreso correctamente a https://www.stanleyworks.es/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC&page=5

Ingreso correctamente a https://www.stanleyworks.es/search?search=Fatmax&f%5B0%5D=content_type%3Asku&sort_by=title&sort_order=ASC&page=6

Ingreso correctamente a https://ww

In [ ]:
ruta=r'C:\Users\SSN0609\OneDrive - Stanley Black & Decker\Dashboards LAG\Proyects\WebScrapping\sku_links2.xlsx'
df_link_skuP1=pd.read_excel(ruta,header=0)

In [ ]:
len(df_link_skuP2)

### Tratamiento de los resultados

In [ ]:
def brands(link):
    brand = ''
    if 'www.' in link:
        brand = link[link.find('www.') + 4:link.find('.com')]
    else:
        brand = link[11:link.find('.global')]
    return brand

def sku(link):
    sku = ''
    if 'product/' in link:
        posStart = link.find('product/') + len('product/')
    else:
        posStart = link.find('producto/') + len('producto/')
    
    posEnd = link.find('/', posStart)
    if posEnd == -1:
        sku = link[posStart:]
    else:
        sku = link[posStart:posEnd]
    return sku

def description(link):
    posEnd = link.rfind('/') + 1
    descripcion = link[posEnd:]
    return descripcion

In [ ]:
df_link_skuP1.columns = df_link_skuP1.columns.str.strip()
df_link_skuP1.columns

In [ ]:
df_link_skuP1['brand'] = df_link_skuP1['link_producto'].apply(brands)
df_link_skuP1['Sku'] = df_link_skuP1['link_producto'].apply(sku)
df_link_skuP1['Descripcion'] = df_link_skuP1['link_producto'].apply(description)
df_link_skuP1.head()

In [ ]:
df_link_skuP1.to_excel(ruta)

In [ ]:
df_link_skuP1['brand'].unique()

In [ ]:
urls_base2=[
    #Black decker
    'https://cl.blackanddecker.global/productos/herramientas-electricas',
    'https://cl.blackanddecker.global/productos/accesorios',
    'https://cl.blackanddecker.global/productos/limpieza-del-hogar',
    'https://cl.blackanddecker.global/productos/exteriores',

    'https://br.blackanddecker.global/produtos/ferramentas-eletricas',
    'https://br.blackanddecker.global/produtos/acessorios',
    'https://br.blackanddecker.global/produtos/ferramentas-manuais',
    'https://br.blackanddecker.global/produtos/manutencao-de-area-externa',

    'https://mx.blackanddecker.global/productos/herramientas-electricas',
    'https://mx.blackanddecker.global/productos/accesorios',
    'https://mx.blackanddecker.global/productos/herramientas-manuales/lasers-e-instrumentos/lasers',
    'https://mx.blackanddecker.global/productos/espacio-de-trabajo',
    'https://mx.blackanddecker.global/productos/espacio-de-trabajo',
    'https://mx.blackanddecker.global/productos/exteriores',

    'https://pe.blackanddecker.global/productos/herramientas-electricas',
    'https://pe.blackanddecker.global/productos/accesorios',
    'https://pe.blackanddecker.global/productos/espacio-de-trabajo',
    'https://pe.blackanddecker.global/productos/exteriores'
    
    #Stanley
    'https://co.stanleytools.global/productos/herramientas-electricas',
    'https://co.stanleytools.global/productos/herramientas-manuales',
    'https://co.stanleytools.global/productos/accesorios',
    'https://co.stanleytools.global/productos/almacenamiento',

    'https://mx.stanleytools.global/productos/herramientas-electricas',
    'https://mx.stanleytools.global/productos/herramientas-manuales',
    'https://mx.stanleytools.global/productos/accesorios',
    'https://mx.stanleytools.global/productos/almacenamiento',
    'https://mx.stanleytools.global/productos/espacio-de-trabajo',
    'https://mx.stanleytools.global/productos/exteriores',

    'https://br.stanleytools.global/produtos/ferramentas-eletricas',
    'https://br.stanleytools.global/produtos/ferramentas-manuais',
    'https://br.stanleytools.global/produtos/acessorios',
    'https://br.stanleytools.global/produtos/armazenamento',
    'https://br.stanleytools.global/produtos/solucoes-para-area-de-trabalho',

    'https://ar.stanleytools.global/productos/herramientas-electricas',
    'https://ar.stanleytools.global/productos/herramientas-manuales',
    'https://ar.stanleytools.global/productos/accesorios',
    'https://ar.stanleytools.global/productos/almacenamiento',
    'https://ar.stanleytools.global/productos/exteriores'



    #Dewalt
    'https://co.dewalt.global/productos/herramientas-electricas',
    'https://co.dewalt.global/productos/accesorios',
    'https://co.dewalt.global/productos/herramientas-manuales',
    'https://co.dewalt.global/productos/almacenaje',
    'https://co.dewalt.global/productos/espacio-de-trabajo',
    
    'https://pe.dewalt.global/productos/herramientas-electricas',
    'https://pe.dewalt.global/productos/accesorios',
    'https://pe.dewalt.global/productos/herramientas-manuales',
    'https://pe.dewalt.global/productos/almacenaje',
    'https://pe.dewalt.global/productos/espacio-de-trabajo',

    'https://br.dewalt.global/produtos/ferramentas-eletricas',
    'https://br.dewalt.global/produtos/acessorios',
    'https://br.dewalt.global/produtos/ferramentas-manuais',
    'https://br.dewalt.global/produtos/manutencao-de-area-externa',
    'https://br.dewalt.global/produtos/armazenamento',
    
    'https://ar.dewalt.global/productos/herramientas-electricas',
    'https://ar.dewalt.global/productos/accesorios',
    'https://ar.dewalt.global/productos/herramientas-manuales',
    'https://ar.dewalt.global/productos/exteriores',
    'https://ar.dewalt.global/productos/area-de-trabajo',
    
    'https://mx.dewalt.global/productos/herramientas-electricas',
    'https://mx.dewalt.global/productos/accesorios',
    'https://mx.dewalt.global/productos/herramientas-manuales',
    'https://mx.dewalt.global/productos/almacenamiento',
    'https://mx.dewalt.global/productos/espacio-de-trabajo',
    
    #Craftsman
    'https://www.craftsman.com/pages/ace-hardware-promotions',
    'https://www.craftsman.com/pages/lowes-promotions',
    'https://www.craftsman.com/collections/power-tools',



#urls_base3=[
    #Black Decker
    'https://ar.blackanddecker.global/productos/herramientas-electricas',
    'https://ar.blackanddecker.global/productos/accesorios',
    'https://ar.blackanddecker.global/productos/espacio-de-trabajo',
    'https://ar.blackanddecker.global/productos/exteriores',

    #Craftsman
    'https://www.craftsman.com/collections/batteries-chargers',
    'https://www.craftsman.com/collections/drills-and-impacts',
    'https://www.craftsman.com/collections/drill-drivers',
    'https://www.craftsman.com/collections/hammer-drills',

    'https://www.craftsman.com/collections/oscillating-tools',
    'https://www.craftsman.com/collections/power-tools',
    'https://www.craftsman.com/collections/reciprocating-saws',
    'https://www.craftsman.com/collections/circular-saws',

    'https://www.craftsman.com/collections/jig-saws',
    'https://www.craftsman.com/collections/miter-saws',
    'https://www.craftsman.com/collections/chop-saws',
    'https://www.craftsman.com/collections/tile-saws',
    
    'https://www.craftsman.com/collections/woodworking-1',
    'https://www.craftsman.com/collections/planers',
    'https://www.craftsman.com/collections/jointers',
    'https://www.craftsman.com/collections/routers',

    'https://www.craftsman.com/collections/sanders',
    'https://www.craftsman.com/collections/polishers',
    'https://www.craftsman.com/collections/grinders',
    'https://www.craftsman.com/collections/nailers-and-compressors',

    'https://www.craftsman.com/collections/brad-nailers',
    'https://www.craftsman.com/collections/framing-nailers',
    'https://www.craftsman.com/collections/finish-nailers',
    'https://www.craftsman.com/collections/specialty-tools',
    'https://www.craftsman.com/collections/accessories-parts-power-tools'
]

# Lista de URLs base de las cuales se navegarán sus páginas para extraer información
urls_base= [
    #Dewalt
    'https://www.dewalt.com/products/accessories',
    'https://www.dewalt.com/products/power-tools',
    'https://www.dewalt.com/products/hand-tools',
    'https://www.dewalt.com/products/outdoor',
    'https://www.dewalt.com/products/workspace/protective-workwear',
    'https://www.dewalt.com/products/storage',
    'https://www.dewalt.com/products/workspace',

    #Stanley
    'https://www.stanleytools.com/products/accessories',
    'https://www.stanleytools.com/products/hand-tools',
    'https://www.stanleytools.com/products/outdoor',
    'https://www.stanleytools.com/products/power-tools',
    'https://www.stanleytools.com/products/storage',
    'https://www.stanleytools.com/products/workspace',
    
    #Facom
    'https://www.facom.com/products/general-tooling',
    'https://www.facom.com/products/profession-tools',
    'https://www.facom.com/products/workstation-and-storage',

    #Proto
    'https://www.protoindustrial.com/products/accessories',
    'https://www.protoindustrial.com/products/hand-tools',
    'https://www.protoindustrial.com/products/power-tools',
    'https://www.protoindustrial.com/products/storage',
    'https://www.protoindustrial.com/products/workspace',
    
    #Irwin
    'https://www.irwintools.com/products/accessories',
    'https://www.irwintools.com/products/hand-tools',
    'https://www.irwintools.com/products/storage',
    'https://www.irwintools.com/products/workspace',

    #Craftsman
    'https://www.craftsman.com/collections/power-tools',
    'https://www.craftsman.com/collections/hand-tools-and-storage',
    'https://www.craftsman.com/collections/hand-tools-and-storage',

    #Lenox
    'https://www.cutwithlenox.com/products/accessories',
    'https://www.cutwithlenox.com/products/hand-tools',
    'https://www.cutwithlenox.com/products/storage',

    #Porter Cable
    'https://www.portercable.com/products/power-tools',
    'https://www.portercable.com/products/accessories',

    #Black Decker
    'https://co.blackanddecker.global/productos/herramientas-electricas',
    'https://co.blackanddecker.global/productos/accesorios',
    'https://co.blackanddecker.global/productos/exteriores'
    ]


#No paginado
[ #Black + Decker
    'https://www.blackanddecker.com/collections/kitchen',
    'https://www.blackanddecker.com/collections/cleaning',
    'https://www.blackanddecker.com/collections/outdoor',
    'https://www.blackanddecker.com/collections/power-tools',
    'https://www.blackanddecker.com/collections/home-lifestyle',
    'https://www.blackanddecker.com/collections/best-sellers',
    'https://www.blackanddecker.com/collections/sale',
    'https://www.blackanddecker.com/collections/gift-finder']
